## Setup

In [ ]:
import mne
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP
import scipy.io
import numpy as np
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
import matplotlib.pyplot as plt

## Parameters

In [ ]:
subject_id = 1
t_start = -0.5
t_end = 2.5
n_ica_components = 5

## Load data

In [ ]:
raw = mne.io.read_raw_gdf(f'./data/train/A0{subject_id}T.gdf', preload=True)
print(raw.info)

In [ ]:
events, event_id = mne.events_from_annotations(raw)
print(event_id)

print(events)

## Define epochs

In [ ]:
# Define the mapping from event code to label
event_dict = {
    'left_hand': event_id['769'], 
    'right_hand': event_id['770'], 
    'feet': event_id['771'], 
    'tongue': event_id['772']
    }

# Create epochs from -0.5s before cue to 4s after
epochs = mne.Epochs(
    raw, 
    events, 
    event_id=event_dict,
    tmin=t_start, tmax=t_end,  # adjust as needed
    baseline=None, # baseline correction
    preload=True
)


In [ ]:
# epochs[0].plot(scalings='auto')

## Remove artifacts

In [ ]:
eog_channels = [ch for ch in raw.ch_names if 'EOG' in ch]

raw.set_channel_types({ch: 'eog' for ch in eog_channels})

# Pick only EEG and EOG channels for ICA
raw_for_ica = raw.copy().pick_types(eeg=True, eog=True)

# Filter for ICA (1Hz high-pass is recommended)
raw_for_ica.filter(l_freq=1.0, h_freq=None)

# Set up and fit ICA
ica = mne.preprocessing.ICA(n_components=n_ica_components,
                            random_state=97, 
                            max_iter='auto')
ica.fit(raw_for_ica)

# Find EOG components automatically
eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name=eog_channels)

# Exclude the found EOG components
ica.exclude = eog_indices

# Now, apply this "cleaned" ICA to your original epochs
# This will remove the EOG artifacts from the EEG data
ica.apply(epochs)

In [ ]:
# epochs[0].plot(scalings='auto')

In [ ]:
eog_channels = [ch for ch in epochs.ch_names if 'EOG' in ch]

epochs.set_channel_types({ch: 'eog' for ch in eog_channels})


In [ ]:
# Filter the epochs in the motor-imagery bands
epochs.filter(l_freq=8.0, h_freq=30.0)

In [ ]:
epochs_eeg = epochs.copy().pick_types(eeg=True, eog=False)

epochs_eeg

## Get features and labels

In [ ]:
X = epochs_eeg.get_data() # Shape (n_trials, n_channels, n_samples)
y = epochs_eeg.events[:, 2] # The event IDs (e.g., 769, 770...)

In [ ]:
# 1. Set up the MNE/SKLearn pipeline
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
lda = LinearDiscriminantAnalysis()
clf = Pipeline([('CSP', csp), ('LDA', lda)])

## Visualize data

In [ ]:
# %matplotlib widget

# # --- 1. Extract CSP Features ---
# csp_viz = CSP(n_components=4, reg=None, log=True, norm_trace=False)
# X_features = csp_viz.fit_transform(X, y)

# # --- 2. Apply LDA for Dimensionality Reduction ---
# lda_viz = LinearDiscriminantAnalysis(n_components=3)

# # Fit and transform the CSP features.
# # This finds the 3 best axes for separating the 4 classes.
# X_3d = lda_viz.fit_transform(X_features, y)

# # --- 3. Plot in 3D ---
# print("Plotting 3D visualization...")

# # Create the mapping from event ID to a name (for the legend)
# label_names = {
#     7: 'Left Hand',
#     8: 'Right Hand',
#     9: 'Feet',
#     10: 'Tongue'
# }
# colors = {
#     7: 'r',
#     8: 'b',
#     9: 'g',
#     10: 'k'
# }


# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(111, projection='3d')

# # Plot each class separately
# for label_val, name in label_names.items():
#     # Find the indices of all trials belonging to this class
#     idx = (y == label_val)
    
#     # Plot only those trials
#     ax.scatter(
#         X_3d[idx, 0],  # 1st component
#         X_3d[idx, 1],  # 2nd component
#         X_3d[idx, 2],  # 3rd component
#         c=colors[label_val],
#         label=name,
#         s=50,          # marker size
#         alpha=0.6
#     )

# ax.set_title('3D Visualization of Motor Imagery Classes (CSP + LDA)')
# ax.set_xlabel('LDA Component 1')
# ax.set_ylabel('LDA Component 2')
# ax.set_zlabel('LDA Component 3')
# ax.legend()
# plt.savefig('lda_3d_visualization.png')

# print("Plot saved as 'lda_3d_visualization.png'")

## Fit model

In [ ]:
clf.fit(X, y)

print("Model trained!")

## Evaluate

### Load eval data

In [ ]:
raw_eval = mne.io.read_raw_gdf(f'./data/eval/A0{subject_id}E.gdf', preload=True)

events_eval, event_id_eval = mne.events_from_annotations(raw_eval)

print(event_id_eval)

### Process eval data

In [ ]:
# 1. Set channel types (same as before)
eog_channels_eval = [ch for ch in raw_eval.ch_names if 'EOG' in ch]
raw_eval.set_channel_types({ch: 'eog' for ch in eog_channels_eval})

# 2. Apply the *fitted* ICA
# (Replace 'ica' with the variable name of your fitted ICA object)
ica.apply(raw_eval) 

# 3. Get events
events_eval, event_id_eval = mne.events_from_annotations(raw_eval)

# 4. Define the event dictionary
# We assume the event codes (e.g., '769') are the same
event_dict_eval = {
    'unknown_cue': event_id_eval['783']
}

# 5. Create epochs (use identical parameters)
epochs_eval = mne.Epochs(
    raw_eval, 
    events_eval, 
    event_id=event_dict_eval,
    tmin=t_start, tmax=t_end,       # Identical tmin/tmax
    baseline=None,  # Identical baseline
    preload=True
)

# 6. Filter (identical frequencies)
epochs_eval.filter(l_freq=8.0, h_freq=30.0)

# 7. Pick only EEG channels
epochs_eval_eeg = epochs_eval.copy().pick_types(eeg=True, eog=False)

### Predict labels

In [ ]:
# Get the preprocessed evaluation data
X_eval = epochs_eval_eeg.get_data()

# Predict using your *trained* classifier
y_pred = clf.predict(X_eval)

label_map = {
    event_id['769']: 1,  # left_hand
    event_id['770']: 2,  # right_hand
    event_id['771']: 3,  # feet
    event_id['772']: 4   # tongue
}

y_pred_mapped = np.array([label_map[pred] for pred in y_pred])

### Load true labels

In [ ]:
# Load the .mat file
mat_labels = scipy.io.loadmat('./data/true_labels/A01E.mat')

print("Keys in .mat file:", mat_labels.keys())

In [ ]:
true_label_key = 'classlabel' 
y_true = mat_labels[true_label_key]

y_true = np.squeeze(y_true)

print(f"Loaded {len(y_true)} true labels.")

### Calculate performance

In [ ]:
# Prediciton performance 

acc = accuracy_score(y_true, y_pred_mapped)

print(f"Final Accuracy: {acc * 100:.2f}%")

# 2. Calculate Kappa (the competition metric)
kappa = cohen_kappa_score(y_true, y_pred_mapped)
print(f"Final Cohen's Kappa: {kappa:.3f}")

# 3. Show a confusion matrix
cm = confusion_matrix(y_true, y_pred_mapped)
print("\nConfusion Matrix:")
print(cm)

In [ ]:
# Random baseline performance 

y_pred_random = np.random.randint(1, 5, size=len(y_true))

acc = accuracy_score(y_true, y_pred_random)

print(f"Final Accuracy: {acc * 100:.2f}%")

# 2. Calculate Kappa (the competition metric)
kappa = cohen_kappa_score(y_true, y_pred_random)
print(f"Final Cohen's Kappa: {kappa:.3f}")

# 3. Show a confusion matrix
cm = confusion_matrix(y_true, y_pred_random)
print("\nConfusion Matrix:")
print(cm)